In [ ]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# 그래프 기본 설정
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False
import matplotlib as mpl
import matplotlib.font_manager as fm

# 폰트 파일 경로 지정
font_path = '/content/drive/MyDrive/font/NanumGothic.ttf'

# 폰트 등록 및 matplotlib 설정
font_name = fm.FontProperties(fname=font_path).get_name()
fm.fontManager.addfont(font_path)
mpl.rc('font', family=font_name)
# 경고 뜨지 않게
import warnings
warnings.filterwarnings('ignore')

#저장라이브러리
import pickle

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 표준화
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# LabelEncoder
from sklearn.preprocessing import LabelEncoder

# 데이터를 학습용과 검증용으로 나눈다.
from sklearn.model_selection import train_test_split

import os

In [ ]:
top_one_df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/주력_비주력_이벤트타임추가버전/one_buyers.parquet') # 주력상품 1회 구매고객 데이터
top_multi_df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/주력_비주력_이벤트타임추가버전/repeat_buyers.parquet') # 주력상품 2회 이상 구매고객 데이터

In [ ]:
df_other = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/other_categories.parquet') # 비주력 상품 데이터
df_top3 = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/top3_categories.parquet') # 주력 상품 데이터

In [ ]:
notop_one_df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/주력_비주력_이벤트타임추가버전/noTop3_one_buyers.parquet') # 비주력상품 1회 구매고객
notop_multi_df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/주력_비주력_이벤트타임추가버전/noTop3_repeat_buyers.parquet') # 비주력상품 2회 이상 구매고객 데이터

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/층화샘플링/top3_categories_750.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/층화샘플링/non_categories_750.csv')

In [ ]:
top_one_df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_date,event_hour,category_lvl1,category_lvl2,category_lvl3,category_lvl4,weekday
0,2019-10-01 05:00:35,view,3701244,2053013565983425517,appliances.environment.vacuum,elenberg,33.44,515535834,d90a6f2f-0d0f-47cf-9fe9-62db93dffdb9,2019-10-01,5,appliances,environment,vacuum,None,Tuesday
1,2019-10-01 05:00:36,view,3600575,2053013563810775923,appliances.kitchen.washer,hotpoint-ariston,275.37,554754045,bd0302ef-c5ca-4b6a-b916-95cc2840c72c,2019-10-01,5,appliances,kitchen,washer,None,Tuesday
2,2019-10-01 05:01:04,view,1004870,2053013555631882655,electronics.smartphone,samsung,286.86,516489361,7d6d03fb-39d6-4143-8694-981a550acaaa,2019-10-01,5,electronics,smartphone,None,None,Tuesday
3,2019-10-01 05:01:07,view,27700113,2053013560086233771,construction.tools.pump,leo,48.90,515630204,f9cc0313-5572-4894-a4eb-45d855f064cc,2019-10-01,5,construction,tools,pump,None,Tuesday
4,2019-10-01 05:01:31,view,3900774,2053013552326770905,appliances.environment.water_heater,artel,72.85,513664804,c0177eae-2279-4b2f-8bf7-e9837cc0612a,2019-10-01,5,appliances,environment,water_heater,None,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28371233,2020-03-31 23:59:54,purchase,6800658,2053013555220840837,appliances.kitchen.juicer,kingston,53.77,624595867,f71468a4-7ca7-453b-954f-f5e72cba556a,2020-03-31,23,appliances,kitchen,juicer,None,Tuesday
28371234,2020-03-31 23:59:54,view,1005098,2232732093077520756,construction.tools.light,samsung,145.18,634646276,41e61625-2646-4b06-b9b0-9b4cc13c9959,2020-03-31,23,construction,tools,light,None,Tuesday
28371235,2020-03-31 23:59:55,view,5800861,2232732082390433922,electronics.audio.subwoofer,kicx,36.55,628330088,ad624635-dd1b-480a-867a-2fda5c0a23a2,2020-03-31,23,electronics,audio,subwoofer,None,Tuesday
28371236,2020-03-31 23:59:56,view,100039953,2232732099754852875,appliances.personal.massager,kivi,257.12,549607680,d92aa32a-2a69-4d97-8259-352bd181806a,2020-03-31,23,appliances,personal,massager,None,Tuesday


In [ ]:
def calculate_path_conversion_rates(df):
    """
    필요 컬럼: ['user_id', 'event_type', 'event_time']
    (동시간대 우선순위를 쓴 경우 'etype_order'가 있으면 함께 정렬)
    반환: 두 경로의 전환율(dict)
    """
    sort_cols = ['user_id', 'event_time']
    if 'etype_order' in df.columns:
        sort_cols.append('etype_order')
    df = df.sort_values(sort_cols)

    # 유저별 이벤트 시퀀스(시간순)
    user_events = df.groupby('user_id')['event_type'].apply(list)

    # 분모: 'view'를 가진 유저 수 (경로의 시작점 통일)
    total_with_view = sum(('view' in ev) for ev in user_events)

    vcp = 0          # view → cart → purchase
    vp_direct = 0    # view → purchase (cart 전혀 없음)

    for ev in user_events:
        if 'view' not in ev:
            continue

        # 1) view → cart → purchase (정확한 순서, 모두 존재)
        if ('cart' in ev) and ('purchase' in ev):
            if ev.index('view') < ev.index('cart') < ev.index('purchase'):
                vcp += 1

        # 2) view → purchase (cart 없음, 직행)
        if ('purchase' in ev) and ('cart' not in ev):
            if ev.index('view') < ev.index('purchase'):
                vp_direct += 1

    rate_vcp = vcp / total_with_view if total_with_view else 0.0
    rate_vp_direct = vp_direct / total_with_view if total_with_view else 0.0

    return {
        'view→cart→purchase': rate_vcp,
        'view→purchase(직행, cart없음)': rate_vp_direct,
        '분모(뷰 보유 유저 수)': int(total_with_view),
        '분자_vcp': int(vcp),
        '분자_vp_direct': int(vp_direct),
    }


view→cart→purchase: 90.67%
view→purchase(직행, cart없음): 7.09%
분모(뷰 보유 유저 수): 687495
분자_vcp: 623336
분자_vp_direct: 48720


In [ ]:
def calculate_path_conversion_rates(df):
    """
    필요 컬럼: ['user_id', 'event_type', 'event_time']
    (동시간대 우선순위를 쓴 경우 'etype_order'가 있으면 함께 정렬)

    반환:
      - 세 경로의 '개수'와 '비율' (분모 = view 보유 유저 수)
      - 기타 내 세부(참고): 미구매, 장바구니까지만
    """
    # 1) 시간순 정렬
    sort_cols = ['user_id', 'event_time']
    if 'etype_order' in df.columns:
        sort_cols.append('etype_order')
    df = df.sort_values(sort_cols)

    # 2) 유저별 이벤트 시퀀스(시간순)
    user_events = df.groupby('user_id')['event_type'].apply(list)

    # 3) 분모: view 보유 유저
    total_with_view = sum(('view' in ev) for ev in user_events)

    # 카운트
    vcp = 0            # view → cart → purchase
    vp_direct = 0      # view → purchase (cart 없음, 직행)
    other = 0          # 기타(미전환/다른 경로)

    # 참고용 세부
    no_purchase = 0    # view 보유 & purchase 없음
    cart_only = 0      # view → cart만 있고 purchase 없음 (순서 v < c)

    for ev in user_events:
        if 'view' not in ev:
            continue

        has_cart = ('cart' in ev)
        has_purchase = ('purchase' in ev)

        iv = ev.index('view')
        ic = ev.index('cart') if has_cart else None
        ip = ev.index('purchase') if has_purchase else None

        # 1) v → c → p (정확한 순서)
        if has_cart and has_purchase and iv < ic < ip:
            vcp += 1

        # 2) v → p 직행 (cart 전혀 없음)
        elif has_purchase and (not has_cart) and iv < ip:
            vp_direct += 1

        # 3) 기타
        else:
            other += 1
            if not has_purchase:
                no_purchase += 1
                if has_cart and iv < ic:
                    cart_only += 1

    # 비율(분모 = view 보유 유저)
    rate_vcp       = (vcp       / total_with_view) if total_with_view else 0.0
    rate_vp_direct = (vp_direct / total_with_view) if total_with_view else 0.0
    rate_other     = (other     / total_with_view) if total_with_view else 0.0

    return {
        '분모(뷰 보유 유저 수)': int(total_with_view),

        # 분자
        '분자_v→c→p': int(vcp),
        '분자_v→p(직행, cart없음)': int(vp_direct),
        '분자_기타(미전환/다른경로)': int(other),

        # 비율 (세 비율 합 = 100%)
        'v→c→p(%)': rate_vcp,
        'v→p_직행(%)': rate_vp_direct,
        '기타(%)': rate_other,

        # 참고용 세부(기타 내부 분해)
        '기타_미구매(개수)': int(no_purchase),
        '기타_장바구니까지만(개수)': int(cart_only),
    }


## 주력 1회구매 고객

In [ ]:
# 예시 실행
path_rates = calculate_path_conversion_rates(top_one_df)
for k, v in path_rates.items():
    if isinstance(v, float):
        print(f"{k}: {v:.2%}")
    else:
        print(f"{k}: {v}")

분모(뷰 보유 유저 수): 687495
분자_v→c→p: 623336
분자_v→p(직행, cart없음): 48720
분자_기타(미전환/다른경로): 15439
v→c→p(%): 90.67%
v→p_직행(%): 7.09%
기타(%): 2.25%
기타_미구매(개수): 0
기타_장바구니까지만(개수): 0


## 주력 다회 구매고객

In [ ]:
path_rates = calculate_path_conversion_rates(top_multi_df)
for k, v in path_rates.items():
    if isinstance(v, float):
        print(f"{k}: {v:.2%}")
    else:
        print(f"{k}: {v}")

분모(뷰 보유 유저 수): 644650
분자_v→c→p: 565483
분자_v→p(직행, cart없음): 10519
분자_기타(미전환/다른경로): 68648
v→c→p(%): 87.72%
v→p_직행(%): 1.63%
기타(%): 10.65%
기타_미구매(개수): 0
기타_장바구니까지만(개수): 0


## 비주력 전체

In [ ]:
path_rates = calculate_path_conversion_rates(df_other)
for k, v in path_rates.items():
    if isinstance(v, float):
        print(f"{k}: {v:.2%}")
    else:
        print(f"{k}: {v}")

분모(뷰 보유 유저 수): 7041465
분자_v→c→p: 453464
분자_v→p(직행, cart없음): 24370
분자_기타(미전환/다른경로): 6563631
v→c→p(%): 6.44%
v→p_직행(%): 0.35%
기타(%): 93.21%
기타_미구매(개수): 6540518
기타_장바구니까지만(개수): 521528


## 주력 전체

In [ ]:
path_rates = calculate_path_conversion_rates(df_top3)
for k, v in path_rates.items():
    if isinstance(v, float):
        print(f"{k}: {v:.2%}")
    else:
        print(f"{k}: {v}")

분모(뷰 보유 유저 수): 9807136
분자_v→c→p: 1188819
분자_v→p(직행, cart없음): 59239
분자_기타(미전환/다른경로): 8559078
v→c→p(%): 12.12%
v→p_직행(%): 0.60%
기타(%): 87.27%
기타_미구매(개수): 8474991
기타_장바구니까지만(개수): 950156


## 비주력 1회 구매 고객

In [ ]:
path_rates = calculate_path_conversion_rates(notop_one_df)
for k, v in path_rates.items():
    if isinstance(v, float):
        print(f"{k}: {v:.2%}")
    else:
        print(f"{k}: {v}")

분모(뷰 보유 유저 수): 295577
분자_v→c→p: 269465
분자_v→p(직행, cart없음): 19970
분자_기타(미전환/다른경로): 6142
v→c→p(%): 91.17%
v→p_직행(%): 6.76%
기타(%): 2.08%
기타_미구매(개수): 0
기타_장바구니까지만(개수): 0


## 비주력 2회 이상 구매 고객

In [ ]:
path_rates = calculate_path_conversion_rates(notop_multi_df)
for k, v in path_rates.items():
    if isinstance(v, float):
        print(f"{k}: {v:.2%}")
    else:
        print(f"{k}: {v}")

분모(뷰 보유 유저 수): 205370
분자_v→c→p: 183999
분자_v→p(직행, cart없음): 4400
분자_기타(미전환/다른경로): 16971
v→c→p(%): 89.59%
v→p_직행(%): 2.14%
기타(%): 8.26%
기타_미구매(개수): 0
기타_장바구니까지만(개수): 0


## 주력 750만 행

In [ ]:
path_rates = calculate_path_conversion_rates(df1)
for k, v in path_rates.items():
    if isinstance(v, float):
        print(f"{k}: {v:.2%}")
    else:
        print(f"{k}: {v}")

분모(뷰 보유 유저 수): 2840521
분자_v→c→p: 14972
분자_v→p(직행, cart없음): 42636
분자_기타(미전환/다른경로): 2782913
v→c→p(%): 0.53%
v→p_직행(%): 1.50%
기타(%): 97.97%
기타_미구매(개수): 2746479
기타_장바구니까지만(개수): 142239


## 비주력 750만 행

In [ ]:
path_rates = calculate_path_conversion_rates(df2)
for k, v in path_rates.items():
    if isinstance(v, float):
        print(f"{k}: {v:.2%}")
    else:
        print(f"{k}: {v}")

분모(뷰 보유 유저 수): 2493133
분자_v→c→p: 12555
분자_v→p(직행, cart없음): 28215
분자_기타(미전환/다른경로): 2452363
v→c→p(%): 0.50%
v→p_직행(%): 1.13%
기타(%): 98.36%
기타_미구매(개수): 2434222
기타_장바구니까지만(개수): 110020


## 서현님 코드 전환율

In [ ]:
def _ensure_event_datetime(df, date_col='event_date', hour_col='event_hour', dt_col='event_time'):
    """
    event_time이 없으면 date+hour로 생성.
    date_col: object/string 날짜
    hour_col: int(hour)
    """
    if dt_col in df.columns:
        return df
    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col], errors='coerce')
    out[dt_col] = out[date_col] + pd.to_timedelta(out[hour_col], unit='h')
    return out

def calc_disjoint_view_purchase_rates(
    df,
    user_col='user_id',
    type_col='event_type',
    dt_col='event_datetime'
):
    """
    분모: view가 있었던 고유 유저 수
    분자1(view_cart_purchase): view < cart < purchase 를 만족하는 유저(최초 view 이후 최초 purchase 사이에 cart가 존재)
    분자2(view_purchase_only): view < purchase 이면서 그 사이에 cart가 전혀 없는 유저
    ※ 두 분자는 서로 겹치지 않도록 엄격 분리. 동시간(같은 hour) 타이는 제외(<, > 사용).
    반환: 각 분자/분모/전환율과 유저 id 집합(원하면 제거 가능)
    """
    # 준비
    df = _ensure_event_datetime(df, dt_col=dt_col).copy()
    df = df.dropna(subset=[dt_col, user_col, type_col])
    # 한 번 정렬(안정 정렬로 재현성 확보; 동시간 타이는 이후 비교식에서 배제)
    df = df.sort_values([user_col, dt_col, type_col], kind='mergesort')

    # 분모: view 있었던 유저들
    view_users = set(df.loc[df[type_col] == 'view', user_col].unique())
    denom = len(view_users)

    vcp_users = set()      # view→cart→purchase
    vp_only_users = set()  # view→purchase (중간 cart 없음)

    # 유저별 타임라인 스캔
    for uid, g in df.groupby(user_col):
        if uid not in view_users:
            continue

        # 이 유저의 이벤트 시퀀스 (시간 오름차순)
        ev = g[[dt_col, type_col]].to_numpy()

        # 1) 첫 view 시각
        t_view = None
        for t, et in ev:
            if et == 'view':
                t_view = t
                break
        if t_view is None:
            continue

        # 2) 첫 view 이후의 '최초 purchase' 시각 (동시각은 제외: t > t_view)
        t_purchase = None
        for t, et in ev:
            if t > t_view and et == 'purchase':
                t_purchase = t
                break
        if t_purchase is None:
            # view는 있었지만 purchase가 없으면 분자 둘 다 미포함
            continue

        # 3) view와 그 '최초 purchase' 사이에 cart가 하나라도 있는지?
        has_cart_between = False
        for t, et in ev:
            if (t > t_view) and (t < t_purchase) and (et == 'cart'):
                has_cart_between = True
                break

        if has_cart_between:
            vcp_users.add(uid)
        else:
            vp_only_users.add(uid)

    # 두 집합이 겹치지 않도록 검증 (설계상 0이어야 함)
    overlap = vcp_users & vp_only_users
    assert len(overlap) == 0, f"분자 집합이 겹칩니다: {len(overlap)}명"

    vcp_num = len(vcp_users)
    vp_only_num = len(vp_only_users)

    rates = {
        'view_cart_purchase': {
            'numerator_users': vcp_users,
            'numerator': vcp_num,
            'denominator': denom,
            'not_converted': max(denom - vcp_num, 0),
            'rate': (vcp_num / denom) if denom else 0.0
        },
        'view_purchase_only': {
            'numerator_users': vp_only_users,
            'numerator': vp_only_num,
            'denominator': denom,
            'not_converted': max(denom - vp_only_num, 0),
            'rate': (vp_only_num / denom) if denom else 0.0
        },
        'denominator_view_users': denom,
        'overlap_check': len(overlap)  # 항상 0이어야 함
    }
    return rates


In [ ]:
# top_one_df = 주력 1회 구매자 데이터셋
# top_multi_df = 주력 2회 이상 구매자 데이터셋

res_once = calc_disjoint_view_purchase_rates(top_one_df)
res_multi = calc_disjoint_view_purchase_rates(top_multi_df)

def pretty_print(title, res):
    print(f"\n[{title}] 분모(view 유저 수) = {res['denominator_view_users']:,}")
    for k in ['view_cart_purchase', 'view_purchase_only']:
        v = res[k]
        print(f"- {k}: 분자={v['numerator']:,} / 분모={v['denominator']:,} "
              f"(미전환={v['not_converted']:,}) → 전환율={v['rate']:.2%}")
    print(f"분자 집합 겹침 수(overlap_check) = {res['overlap_check']} (항상 0이어야 함)")

pretty_print("주력 1회 구매자", res_once)
pretty_print("주력 2회 이상 구매자", res_multi)



[주력 1회 구매자] 분모(view 유저 수) = 687,495
- view_cart_purchase: 분자=153,589 / 분모=687,495 (미전환=533,906) → 전환율=22.34%
- view_purchase_only: 분자=300,297 / 분모=687,495 (미전환=387,198) → 전환율=43.68%
분자 집합 겹침 수(overlap_check) = 0 (항상 0이어야 함)

[주력 2회 이상 구매자] 분모(view 유저 수) = 644,650
- view_cart_purchase: 분자=163,950 / 분모=644,650 (미전환=480,700) → 전환율=25.43%
- view_purchase_only: 분자=434,893 / 분모=644,650 (미전환=209,757) → 전환율=67.46%
분자 집합 겹침 수(overlap_check) = 0 (항상 0이어야 함)


In [ ]:
# top_one_df = 주력 1회 구매자 데이터셋
# top_multi_df = 주력 2회 이상 구매자 데이터셋

res_once = calc_disjoint_view_purchase_rates(df2)
# res_multi = calc_disjoint_view_purchase_rates(top_multi_df)

def pretty_print(title, res):
    print(f"\n[{title}] 분모(view 유저 수) = {res['denominator_view_users']:,}")
    for k in ['view_cart_purchase', 'view_purchase_only']:
        v = res[k]
        print(f"- {k}: 분자={v['numerator']:,} / 분모={v['denominator']:,} "
              f"(미전환={v['not_converted']:,}) → 전환율={v['rate']:.2%}")
    print(f"분자 집합 겹침 수(overlap_check) = {res['overlap_check']} (항상 0이어야 함)")

pretty_print("주력 1회 구매자", res_once)
# pretty_print("주력 2회 이상 구매자", res_multi)



[주력 1회 구매자] 분모(view 유저 수) = 2,493,133
- view_cart_purchase: 분자=7,880 / 분모=2,493,133 (미전환=2,485,253) → 전환율=0.32%
- view_purchase_only: 분자=35,687 / 분모=2,493,133 (미전환=2,457,446) → 전환율=1.43%
분자 집합 겹침 수(overlap_check) = 0 (항상 0이어야 함)


In [ ]:
# top_one_df = 주력 1회 구매자 데이터셋
# top_multi_df = 주력 2회 이상 구매자 데이터셋

res_once = calc_disjoint_view_purchase_rates(df1)
# res_multi = calc_disjoint_view_purchase_rates(top_multi_df)

def pretty_print(title, res):
    print(f"\n[{title}] 분모(view 유저 수) = {res['denominator_view_users']:,}")
    for k in ['view_cart_purchase', 'view_purchase_only']:
        v = res[k]
        print(f"- {k}: 분자={v['numerator']:,} / 분모={v['denominator']:,} "
              f"(미전환={v['not_converted']:,}) → 전환율={v['rate']:.2%}")
    print(f"분자 집합 겹침 수(overlap_check) = {res['overlap_check']} (항상 0이어야 함)")

pretty_print("주력", res_once)
# pretty_print("주력 2회 이상 구매자", res_multi)



[주력] 분모(view 유저 수) = 2,840,521
- view_cart_purchase: 분자=11,970 / 분모=2,840,521 (미전환=2,828,551) → 전환율=0.42%
- view_purchase_only: 분자=54,263 / 분모=2,840,521 (미전환=2,786,258) → 전환율=1.91%
분자 집합 겹침 수(overlap_check) = 0 (항상 0이어야 함)
